In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

In [2]:
working_dir = os.getcwd()

In [3]:
def get_one_contract_data_by_date(date_str, contract_pd):
    """Get the contract minute data by the date
    Trading hours:
        Sunday – Friday, 7:00 p.m. – 7:45 a.m. CT and 
        Monday – Friday, 8:30 a.m. – 1:20 p.m. CT
    Define the last day's night-time and today's day-time as today's trading hours
    
    Args:
        date_str: str, the date we want to extract the main contract data, e.g. "2016-01-04"
        contract_path: str, the main contract path, e.g. "../data/ZC/ZCH16.csv"
        
    Returns:
        pandas.DataFrame
    
    """
    year, month, day = map(int, date_str.split('-'))
    last_date_str = (datetime(year, month, day) - timedelta(days=1)).strftime("%Y-%m-%d")
    
    # all_data = pd.read_csv(contract_path, header=0, index_col=0, parse_dates=[0])
    all_data = contract_pd
    all_data['Volume'] = all_data.TotalVolume.diff()
    
    last_day_data = all_data.loc[last_date_str].between_time("19:00", "23:59")
    today_data = all_data.loc[date_str].between_time("0:00", "13:20")
    
    data = last_day_data.append(today_data, ignore_index=False)
    data.iloc[0, 5] = data.iloc[0, 4] #The first minute's trading volume 
    return data


In [4]:
def extract_data_from_unique_contract(contract_path, unique_dates):
    contract_pd = pd.read_csv(contract_path, header=0, index_col=0, parse_dates=[0])
    data = pd.DataFrame(columns=["Open", "High", "Low", "Close", "TotalVolume", "Volume"])
    for single_date in unique_dates:
        dd = get_one_contract_data_by_date(single_date, contract_pd)
        data = data.append(dd, ignore_index=False)
    return data

In [5]:
def get_all_main_contract_data(main_contracts_path = working_dir+"/data/ZC/front.csv", contracts_root_path = working_dir+"/data/ZC/"):
    """Combine the daily main contract data 
    Args:
        main_contracts_path: str, the path that have the daily main contract code
        contracts_root_path: str, the root path that have all the contract data
        
    Returns:
        data, pd.DataFrame,columns=["Open", "High", "Low", "Close", "TotalVolume", "Volume", "Change"], 
                            where "Change" == 1 indicate that the main contract changed that day
    
    """
    func_contract_to_path = lambda x: contracts_root_path + "ZC" + x + ".csv"
    
    main_contracts = pd.read_csv(main_contracts_path, header=None, names=['date_str', 'main_contract'])
    unique_contracts = main_contracts.main_contract.unique()
    
    data = pd.DataFrame(columns=["Open", "High", "Low", "Close", "TotalVolume", "Volume"])
    for single_contract in unique_contracts:
        contract_path = func_contract_to_path(single_contract)
        unique_dates = main_contracts.loc[main_contracts["main_contract"] == single_contract].date_str.unique()
        today_data = extract_data_from_unique_contract(contract_path, unique_dates)
        data = data.append(today_data, ignore_index=False)
        
    return data

In [6]:
%time all_data = get_all_main_contract_data()

CPU times: user 9.67 s, sys: 1.24 s, total: 10.9 s
Wall time: 11.2 s


In [8]:
all_data.to_csv(working_dir+"/data/combined_data.csv")

In [9]:
len(all_data)

535536

In [10]:
# Get the main contracts changing date
main_contracts = pd.read_csv(working_dir+"/data/ZC/front.csv", header=None, names=['date_str', 'main_contract'])
main_contracts['Change'] = main_contracts.main_contract.apply(lambda x: hash(x)).rolling(2).apply(lambda x: 1 if x[1] != x[0] else 0).fillna(1)
change_dates = main_contracts[main_contracts['Change'] == 1][['date_str', 'main_contract']].reset_index(drop=True)
change_dates


C:\Users\renhao\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until


,date_str,main_contract
0,2016-01-04,H16
1,2016-02-24,K16
2,2016-04-20,N16
3,2016-06-29,U16
4,2016-08-17,Z16
5,2016-11-29,H17
6,2017-02-27,K17
7,2017-04-20,N17
8,2017-06-28,U17
9,2017-08-17,Z17


In [12]:
change_dates.to_csv(working_dir+"/data/main_contract_change_dates.csv", index=False)

,Open,High,Low,Close,TotalVolume,Volume
2016-01-03 19:01:00,359.50,360.00,359.00,359.75,1215,1215.0
2016-01-03 19:02:00,359.75,359.75,359.25,359.50,1715,500.0
2016-01-03 19:03:00,359.50,359.50,359.00,359.00,2059,344.0
2016-01-03 19:04:00,359.00,359.25,359.00,359.00,2077,18.0
2016-01-03 19:05:00,359.00,359.00,359.00,359.00,2119,42.0


In [7]:
def get_sub_data_between_time_interval(all_data,
                                   start_date="2016-01-03", 
                                   start_time_in_day="19:00:00", 
                                   end_date="2019-03-01", 
                                   end_time_in_day="13:20:00",
                                   include = True):
    start_index = start_date + " " + start_time_in_day
    end_index = end_date + " " + end_time_in_day
    t_interval = (all_data.index >= start_index) & (all_data.index <= end_index) \
        if include else \
        (all_data.index > start_index) & (all_data.index < end_index)
    sub_data = all_data.loc[t_interval]
    return sub_data

def get_sub_data_in_no_of_transactions(all_data,
                                   start_date="2016-01-03", 
                                   start_time_in_day="19:00:00", 
                                   no_of_trans = 10000,
                                   include = True):
    start_index = start_date + " " + start_time_in_day
    t_interval = (all_data.index >= start_index) \
        if include else \
        (all_data.index > start_index)
    sub_data = all_data.loc[t_interval][:no_of_trans]
    return sub_data
    
def get_combined_sub_data_by_period(all_data,
                                    start_date="2016-01-03", 
                                    start_time_in_day="19:00:00",
                                    end_date="2019-03-01", 
                                    end_time_in_day="13:20:00",
                                    period = "D",
                                    include = True):
    start_index = start_date + " " + start_time_in_day
    end_index = end_date + " " + end_time_in_day
    t_interval = (all_data.index >= start_index) & (all_data.index <= end_index) \
        if include else \
        (all_data.index > start_index) & (all_data.index < end_index)
    sub_data = all_data.loc[t_interval]
    
    sub_data = sub_data.between_time('19:00', '13:20', include_start=True, include_end=True)
    proxy = sub_data.index + pd.DateOffset(hours=5)
    combined_data = sub_data.groupby(proxy.date).agg(
        {'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'TotalVolume': 'last'})
    combined_data = combined_data.reindex(columns=['Open', 'High', 'Low', 'Close', 'TotalVolume'])
    combined_data.index = pd.DatetimeIndex(combined_data.index)
    return combined_data.resample(period).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'TotalVolume': 'sum'})

In [8]:
def get_stat_val_between_time_interval(sub_data,
                                   attribute="Close", 
                                   stat_method="Mean"):
                                   # start_date="2016-01-03", 
                                   # start_time_in_day="19:00:00", 
                                   # end_date="2019-03-01", 
                                   # end_time_in_day="13:20:00",
                                   # no_of_trans = 10000,
                                   # include = True,
                                   # by_time = True):
    
    # sub_data = get_sub_data_between_time_interval(all_data, start_date, start_time_in_day, end_date, end_time_in_day, include) \
    #     if by_time else get_sub_data_in_no_of_transactions(all_data, start_date, start_time_in_day, no_of_trans, include)
    
    stat_dict = {
        "Mean": sub_data[attribute].mean(),
        "SD": sub_data[attribute].std(),
        "Diff": sub_data[attribute].diff(),
        "P_change": sub_data[attribute].pct_change(),
        "Log_return": np.log(sub_data[attribute]) - np.log(sub_data[attribute].shift(1)),
    }
    
    return stat_dict[stat_method]

2016-01-03 19:02:00         NaN
2016-01-03 19:03:00   -0.001392
2016-01-03 19:04:00    0.000000
2016-01-03 19:05:00    0.000000
2016-01-03 19:06:00    0.000696
2016-01-03 19:07:00   -0.000696
2016-01-03 19:08:00   -0.000697
2016-01-03 19:09:00    0.000697
2016-01-03 19:10:00    0.000000
2016-01-03 19:11:00   -0.000697
2016-01-03 19:13:00    0.000697
2016-01-03 19:14:00    0.000000
2016-01-03 19:17:00   -0.000697
2016-01-03 19:20:00    0.000000
Name: Close, dtype: float64
2016-01-03 19:03:00   -0.001391
2016-01-03 19:04:00    0.000000
2016-01-03 19:05:00    0.000000
2016-01-03 19:06:00    0.000696
2016-01-03 19:07:00   -0.000696
2016-01-03 19:08:00   -0.000696
2016-01-03 19:09:00    0.000697
2016-01-03 19:10:00    0.000000
2016-01-03 19:11:00   -0.000696
2016-01-03 19:13:00    0.000697
2016-01-03 19:14:00    0.000000
2016-01-03 19:17:00   -0.000696
2016-01-03 19:20:00    0.000000
Name: Close, dtype: float64
2016-01-03 19:02:00         NaN
2016-01-03 19:03:00   -0.001392
2016-01-03 19:04

,Open,High,Low,Close,TotalVolume
2016-01-04,359.50,360.0,350.5,352.00,170742
2016-01-05,352.75,354.5,352.5,354.25,8257


In [9]:
# Get all data from what we have, put your combined_data.csv under the folder named "data" in your working directory
all_data = pd.read_csv(working_dir+"/data/combined_data.csv", header=0, index_col=0, parse_dates=[0])

# Get tick data by time interval from 2016-01-03 19:00:00 to 2016-01-05 13:20:00
sample_data_1 = get_sub_data_between_time_interval(all_data, start_date="2016-01-03", start_time_in_day="19:00:00", end_date="2016-01-05", 
                              end_time_in_day="13:20:00")
print(sample_data_1.head())

# Get next 10000 transaction data start from 2016-01-03 19:00:00
sample_data_2 = get_sub_data_in_no_of_transactions(all_data, start_date="2016-01-03", start_time_in_day="19:00:00", no_of_trans=10000)
print(sample_data_2.head())

# Get daily data by time interval from 2016-01-03 19:00:00 to 2016-01-05 13:20:00
sample_data_3 = get_combined_sub_data_by_period(all_data, period="D", start_date="2016-01-03", start_time_in_day="19:00:00", end_date="2016-01-05", 
                               end_time_in_day="13:20:00")
print(sample_data_3.head())

# Get statistic values from sub data
# Mean close price for sample 1
mean_close_1 = get_stat_val_between_time_interval(sample_data_1, attribute="Close", stat_method="Mean")
print(mean_close_1)

# Mean volume for sample 1
mean_volume_1 = get_stat_val_between_time_interval(sample_data_1, attribute="Volume", stat_method="Mean")
print(mean_volume_1)

# Standard deviation of close price for sample 1
sd_close_1 = get_stat_val_between_time_interval(sample_data_1, attribute="Close", stat_method="SD")
print(sd_close_1)

# Change of close price for sample 1
diff_close_1 = get_stat_val_between_time_interval(sample_data_1, attribute="Close", stat_method="Diff")
print(diff_close_1)

# Percentage change of close price for sample 1
pchange_close_1 = get_stat_val_between_time_interval(sample_data_1, attribute="Close", stat_method="P_change")
print(pchange_close_1)

# Log return of close price for sample 1
lr_close_1 = get_stat_val_between_time_interval(sample_data_1, attribute="Close", stat_method="Log_return")
print(lr_close_1)

                       Open    High     Low   Close  TotalVolume  Volume
2016-01-03 19:01:00  359.50  360.00  359.00  359.75         1215  1215.0
2016-01-03 19:02:00  359.75  359.75  359.25  359.50         1715   500.0
2016-01-03 19:03:00  359.50  359.50  359.00  359.00         2059   344.0
2016-01-03 19:04:00  359.00  359.25  359.00  359.00         2077    18.0
2016-01-03 19:05:00  359.00  359.00  359.00  359.00         2119    42.0
                       Open    High     Low   Close  TotalVolume  Volume
2016-01-03 19:01:00  359.50  360.00  359.00  359.75         1215  1215.0
2016-01-03 19:02:00  359.75  359.75  359.25  359.50         1715   500.0
2016-01-03 19:03:00  359.50  359.50  359.00  359.00         2059   344.0
2016-01-03 19:04:00  359.00  359.25  359.00  359.00         2077    18.0
2016-01-03 19:05:00  359.00  359.00  359.00  359.00         2119    42.0
              Open   High     Low  Close  TotalVolume
2016-01-04  359.50  360.0  350.50  352.0       170742
2016-01-05  352.